# Instruções para o uso do Notebook

Para rodar o notebook a seguir corretamente, é necessário seguir alguns passos prévios. Primeiro, baixe os datasets disponíveis no link a seguir: [Datasets](https://drive.google.com/drive/folders/1bm6iQenyZ63gbw_s7j0md8UaHP84O0Vn?usp=sharing) (não é necessário todos, a princípio, somente os citados abaixo). Esses datasets não estão incluídos no notebook devido ao seu tamanho, por isso são ignorados pelo arquivo `.gitignore`. Todos eles devem ser colocados na pasta [`data`](data), localizada em `src/Notebook/Api_Sprint_4/data`.

Os arquivos necessários são:

- **GoogleNews-vectors-negative300.bin**: [Download](https://drive.google.com/open?id=1IzDDngIEWzGP4onWlBCglAQ87g5gP2CL&usp=drive_copy) - contém palavras pré-treinadas em 300 dimensões.
- **classification-labeled.csv**: [Download](https://drive.google.com/file/d/1AjRl3mKWceHjTFEZsHUrAwCRs-12UQQj/view?usp=drive_link) - contém o banco de dados original, entregue pela *Uber*.
- **processed_text_data.csv**: [Download](https://drive.google.com/file/d/1hCW3pliSKDWKyr0h0tvNBtUgtw9EsE3c/view?usp=sharing) - contém o banco de dados processado.
- **data_for_training_models.csv**: [Download](https://drive.google.com/file/d/1yJIvk_He-zeB9x6LDXhv0cAll5Tzwla9/view?usp=drive_link) - contém as palavras do banco de dados em vetores numéricos (a etapa de desenvolvimento do Word2Vec pode ser acompanhado no arquivo [Encapsulamento.ipynb](Encapsulamento.ipynb)).

Todos são necessários para rodar as células seguintes.

Outras recomendações:

- Mantenha o C++ do seu computador atualizado, pois determinadas bibliotecas são compiladas com essa linguagem.
- Caso queira conferir as versões mais atualizaas das bibliotecas, elas estão detalhadas no aquivo [``requirements.txt``](../../requirements.txt). Para usar este arquivo, basta rodar no terminal o comando: ``pip install -r requirements.txt``. Porém, a princípio, é possível ter acesso a todas as bilioteca ao rodar a célula de "Instalação e Importação das Bibliotecas" deste Notebook.

## Bibliotecas

A utilização de diversas bibliotecas é essencial para a implementação de pipelines de processamento de linguagem natural (PLN) e aprendizado de máquina. Neste documento, destacamos as bibliotecas utilizadas em nossos projetos e suas respectivas funções.


In [ ]:
# Instalação de "Bibliotecas Mágicas"
%pip install pandas
%pip install numpy
%pip install nltk
%pip install emoji
%pip install wordcloud
%pip install emot
%pip install imbalanced-learn
%pip install matplotlib
%pip install scikit-learn
%pip install spacy
!python -m spacy download en_core_web_md
%pip install scipy==1.11
%pip install gensim
%pip install scikit-learn
%pip install seaborn
%pip install xgboost

In [ ]:
# Importação de Bibliotecas
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix, roc_curve, roc_auc_score
import tempfile
import os
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
from sklearn.svm import SVC
from sklearn.preprocessing import label_binarize
from sklearn.preprocessing import StandardScaler
from itertools import cycle
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

In [ ]:
# Importação de Bibliotecas
import pandas as pd
import numpy as np
from textblob import TextBlob
import emoji
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import word_tokenize
from emot.emo_unicode import UNICODE_EMOJI  
import xgboost as xgb
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

In [ ]:
import re
import pickle
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from textblob import TextBlob
from xgboost import DMatrix
import emoji

## Pré-processamento



O pré-processamento de texto é uma etapa crucial em projetos de análise de sentimentos e processamento de linguagem natural (PLN). Ele transforma texto bruto em um formato limpo e estruturado, melhorando a qualidade dos dados para modelos de aprendizado de máquina.

Este pipeline de pré-processamento foi desenvolvido para tratar textos em inglês e inclui as seguintes etapas:

1. **Conversão de Emojis em Palavras:** Utiliza a biblioteca `emoji` para converter emojis em descrições textuais, capturando emoções e sentimentos expressos por símbolos.
2. **Limpeza do Texto:** Remove URLs, tags HTML, caracteres especiais e números, e corrige espaços extras.
3. **Correção Ortográfica:** Usa a biblioteca `TextBlob` para corrigir erros ortográficos, preservando palavras específicas que não devem ser modificadas.
4. **Tokenização:** Divide o texto em palavras individuais (tokens).
5. **Remoção de Stopwords:** Filtra palavras comuns que geralmente não contribuem para o significado principal do texto.

Essas etapas garantem que os dados textuais estejam prontos para serem usados em modelos de aprendizado de máquina, melhorando a precisão e a eficácia das análises.


In [ ]:
# Função para converter emojis em palavras
def convert_emojis(text):
    return emoji.demojize(text)

# Função para limpar o texto
def clean_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'<[^>]+>', '', text)  # Remove tags HTML
    text = re.sub(r'[^a-zA-Z\s]', '', text)  # Remove caracteres especiais e números
    text = re.sub(r'\s+', ' ', text).strip()  # Corrigir espaços extras
    return text

# Função para tokenizar o texto
def tokenize(text):
    tokens = word_tokenize(text.lower())  # Convertendo para minúsculas e tokenizando
    return tokens

# Função para remover stopwords
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token not in stop_words]
    return filtered_tokens

# Lista de palavras a serem mantidas
words_to_keep = ['uber']

def correct_spelling(text):
    # Substituir palavras a serem mantidas por placeholders temporários
    for i, word in enumerate(words_to_keep):
        text = text.replace(word, f'PLACEHOLDER_{i}')
    
    # Corrigir o texto
    corrected_text = str(TextBlob(text).correct())
    
    # Restaurar as palavras mantidas
    for i, word in enumerate(words_to_keep):
        corrected_text = corrected_text.replace(f'PLACEHOLDER_{i}', word)
    
    return corrected_text

# Função principal de pré-processamento para uma única frase
def preprocess_text(text):
    text = convert_emojis(text)
    text = clean_text(text)
    text = correct_spelling(text)
    tokens = tokenize(text)
    tokens = remove_stopwords(tokens)
    return ' '.join(tokens)

In [ ]:
nltk.download('punkt')
nltk.download('stopwords')
frase_aleatoria = 'uber is good'
# Aplicando o pré-processamento na frase aleatória
texto_processado = preprocess_text(frase_aleatoria)
print(f"Texto processado: {texto_processado}")

## Vetorização



A vetorização de texto é uma etapa essencial no processamento de linguagem natural (PLN). Ela converte palavras e frases em representações vetoriais permitindo que sejam processadas por modelos de aprendizado de máquina. Utilizar modelos pré-treinados como o Word2Vec permite capturar as relações semânticas entre palavras, o que é crucial para tarefas como análise de sentimentos, tradução automática e reconhecimento de entidades.

No processo de vetorização, um modelo Word2Vec pré-treinado é utilizado para transformar cada palavra em um vetor numérico. Esses vetores representam a semântica das palavras e permitem operações matemáticas para analisar similaridades e diferenças. Este documento descreve o uso de um modelo Word2Vec pré-treinado, especificamente o modelo do Google News, para gerar vetores de palavras a partir de textos, demonstrando como esses vetores podem ser utilizados para representar frases de forma eficaz em um formato vetorial.


In [ ]:
EMBEDDING_FILE = './data/GoogleNews-vectors-negative300.bin'
word_vectors = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

In [ ]:
def prepare_word2vec_vectors(text, embedding_file):
    """
    Gera uma representação vetorial para uma única frase usando um modelo Word2Vec pré-treinado.

    Inputs:
        text (str): Frase que pretende ser vetorizada.
        embedding_file (str): Caminho para o arquivo binário do modelo Word2Vec pré-treinado.

    Output: DataFrame: DataFrame com o vetor de palavras para a frase.
    """
    # Carregar o modelo pré-treinado
    word_vectors = KeyedVectors.load_word2vec_format(embedding_file, binary=True)

    # Inicializar um vetorizador para tokenizar o texto
    vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')  # Tokeniza palavras
    vectorizer.fit([text])  # Ajustar o vetorizador com a frase

    # Inicializar vetor para armazenar o vetor de palavras
    word_embedding = np.zeros((word_vectors.vector_size,))

    tokens = vectorizer.build_tokenizer()(text)
    word_vecs = [word_vectors[word] for word in tokens if word in word_vectors.key_to_index]
    if word_vecs:
        word_embedding = np.sum(word_vecs, axis=0)

    # Criar um DataFrame com o vetor
    return pd.DataFrame([word_embedding])

In [ ]:
text = 'uber is good'

embedding_file = './data/GoogleNews-vectors-negative300.bin'

# Obter o vetor da frase completa usando a função
vector_df_sentence = prepare_word2vec_vectors(text, embedding_file)
full_text_vector = vector_df_sentence.iloc[0].values

# Visualizar a matriz da frase completa 
print(f"Vetor da frase completa '{text}':")

In [ ]:
print(full_text_vector)

In [ ]:
print(vector_df_sentence)

## Treinamento e salvamento do modelo



O treinamento e salvamento de modelos de aprendizado de máquina são etapas fundamentais no desenvolvimento de soluções de inteligência artificial. Este processo envolve preparar os dados, treinar o modelo, avaliar seu desempenho e, por fim, salvar o modelo treinado para uso futuro. 

Neste exemplo, utilizamos o algoritmo XGBoost para treinar um modelo de classificação de sentimentos, utilizando dados vetorizados previamente com Word2Vec. O processo inclui a carga e preparação dos dados, a divisão em conjuntos de treinamento e teste, o treinamento do modelo XGBoost, a avaliação de seu desempenho por meio de várias métricas, e o salvamento do modelo treinado em um arquivo. Esse fluxo de trabalho garante que o modelo esteja pronto para ser aplicado em novos dados de maneira eficiente e precisa.


In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import numpy as np

def train_xgboost_model(file_path, model_save_path):
    """
    Treina um modelo XGBoost utilizando Word2Vec e exibe as métricas de avaliação.

    Inputs:
        file_path (str): Caminho para o arquivo CSV contendo os dados de treinamento.
        model_save_path (str): Caminho para salvar o modelo XGBoost treinado.

    Outputs: 
        dict: Um dicionário contendo as métricas de avaliação do modelo ou None se ocorrer um erro.
    """
    try:
        # Carregar os dados
        data = pd.read_csv(file_path)

        # Verificar a presença da coluna 'sentiment'
        if 'sentiment' not in data.columns:
            raise KeyError("'sentiment' não encontrado nas colunas do DataFrame")

        # Dividir os dados em features (X) e target (y)
        X = data.drop('sentiment', axis=1)
        y = data['sentiment']

        # Verificar os rótulos únicos antes da codificação
        print("Rótulos únicos antes da codificação:", y.unique())

        # Re-rotular os rótulos para garantir que estão no intervalo [0, num_class)
        label_encoder = LabelEncoder()
        y = label_encoder.fit_transform(y)

        # Verificar os rótulos únicos após a codificação
        print("Rótulos únicos após a codificação:", np.unique(y))

        # Dividir os dados em conjuntos de treinamento e teste
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        # Criar o DMatrix para o XGBoost
        dtrain = xgb.DMatrix(X_train, label=y_train)
        dtest = xgb.DMatrix(X_test, label=y_test)

        # Definir os parâmetros do XGBoost
        params = {
            'max_depth': 6,
            'eta': 0.3,
            'objective': 'multi:softmax',
            'num_class': 3
        }

        # Treinar o modelo
        bst = xgb.train(params, dtrain, num_boost_round=100)

        # Salvar o modelo treinado
        with open(model_save_path, 'wb') as file:
            pickle.dump(bst, file)

        # Fazer previsões
        y_pred = bst.predict(dtest)

        # Verificar os rótulos únicos nas previsões
        print("Rótulos únicos nas previsões:", np.unique(y_pred))

        # Calcular e exibir o relatório de classificação
        class_report = classification_report(y_test, y_pred)
        print("Classification Report:\n", class_report)

        # Calcular e visualizar a matriz de confusão
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(10, 8))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", xticklabels=['Negativo', 'Neutro', 'Positivo'], yticklabels=['Negativo', 'Neutro', 'Positivo'])
        plt.title("Matriz de Confusão")
        plt.xlabel("Valor Previsto")
        plt.ylabel("Valor Real")
        plt.show()

        # Calcular métricas
        metrics = {
            'accuracy': accuracy_score(y_test, y_pred),
            'precision': precision_score(y_test, y_pred, average='weighted'),
            'recall': recall_score(y_test, y_pred, average='weighted'),
            'f1_score': f1_score(y_test, y_pred, average='weighted')
        }

        return metrics

    except Exception as e:
        print(f"Ocorreu um erro: {e}")
        return None


In [ ]:
# Exemplo de uso
file_path = './data/data_for_training_models.csv'
model_save_path = './models/xgboost_model.pkl'
metrics = train_xgboost_model(file_path, model_save_path)
print("Métricas:", metrics)

## Modelo



A classificação de texto em sentimentos positivos, negativos ou neutros é uma aplicação comum em processamento de linguagem natural (PLN). Para realizar essa tarefa, é possível utilizar um modelo de aprendizado de máquina previamente treinado, como o XGBoost. 

Nesta abordagem, a função `classify_text` é responsável por classificar uma frase com base em seu vetor de características. O modelo XGBoost, treinado anteriormente e salvo em um arquivo, é carregado e utilizado para prever a classe do texto. Esta função facilita a aplicação do modelo em novos dados, proporcionando uma classificação eficiente e precisa de sentimentos.


In [ ]:
# Função para classificar o texto
def classify_text(vectorized_text, model_file):
    """
    Classifica uma frase como positiva (1), negativa (-1) ou neutra (0) usando um modelo XGBoost treinado.

    Inputs:
        vectorized_text (np.ndarray): Vetor representando a frase vetorizada.
        model_file (str): Caminho para o arquivo pickle contendo o modelo XGBoost treinado.

    Output: int: Classificação da frase (1: positiva, -1: negativa, 0: neutra).
    """
    # Carregar o modelo XGBoost treinado
    with open(model_file, 'rb') as file:
        model = pickle.load(file)

    # Criar um DMatrix a partir do vetorizado texto
    feature_names = [str(i) for i in range(vectorized_text.shape[1])]
    dmatrix = xgb.DMatrix(vectorized_text, feature_names=feature_names)

    # Fazer a predição usando o modelo carregado
    prediction = model.predict(dmatrix)
    
    return int(prediction[0])

## Chamada do modelo



Para categorizar uma frase em termos de sentimentos, utilizamos um pipeline que combina pré-processamento, vetorização e classificação. Este pipeline permite transformar texto bruto em uma classificação de sentimento de maneira eficiente e precisa.

A função `categorizing_sentence` integra todas as etapas necessárias para processar e classificar uma frase. Ela pré-processa a frase para remover ruídos, vetoriza a frase utilizando um modelo Word2Vec pré-treinado e, em seguida, classifica o vetor de características utilizando um modelo XGBoost treinado.

Essa abordagem é testada com uma frase de exemplo, demonstrando a aplicação prática do pipeline para determinar o sentimento do texto. Com isso, é possível aplicar esta metodologia para analisar sentimentos em diversos textos em inglês, proporcionando uma ferramenta poderosa para várias aplicações de análise de sentimentos.


In [ ]:
# Função principal para categorizar a frase
def categorizing_sentence(text, embedding_file, model_file):
    """
    Processa e classifica uma frase como positiva (1), negativa (-1) ou neutra (0).

    Inputs:
        text (str): Frase a ser classificada.
        embedding_file (str): Caminho para o arquivo binário do modelo Word2Vec pré-treinado.
        model_file (str): Caminho para o arquivo pickle contendo o modelo XGBoost treinado.

    Output: int: Classificação da frase (1: positiva, -1: negativa, 0: neutra).
    """
    # Pré-processar a frase
    preprocessed_text = preprocess_text(text)
    
    # Vetorizar a frase
    vectorized_text = prepare_word2vec_vectors(preprocessed_text, embedding_file)
    
    # Classificar a frase
    classification = classify_text(vectorized_text.values, model_file)
    
    return classification

In [ ]:
# Teste com uma frase de exemplo
text = "i love uber!"
embedding_file = './data/GoogleNews-vectors-negative300.bin'
model_file = './models/xgboost_model.pkl'

result = categorizing_sentence(text, embedding_file, model_file)
print("Classificação:", result)

# Análise Comparativa de Modelos de Classificação de Sentimentos

Nesta seção, apresentamos uma análise comparativa de três modelos de classificação de sentimentos: FastText, TF-IDF e GloVe de 50 dimensões. A análise considera duas categorias de classificação: Negativo vs Não Negativo e Neutro vs Positivo.

## Resultados dos Modelos

### FastText

**Negativo vs Não Negativo**
- **Precision**: 0.94
- **Recall**: 0.94
- **F1-Score**: 0.94
- **Accuracy**: 0.94
- **Kappa**: 0.88
- **Mean Cross-Val F1**: 0.933
- **Std Cross-Val F1**: 0.018

**Neutro vs Positivo**
- **Precision**: 0.97
- **Recall**: 0.97
- **F1-Score**: 0.97
- **Accuracy**: 0.97
- **Kappa**: 0.93
- **Mean Cross-Val F1**: 0.931
- **Std Cross-Val F1**: 0.006

### TF-IDF

**Negativo vs Não Negativo**
- **Precision**: 0.92
- **Recall**: 0.91
- **F1-Score**: 0.91
- **Accuracy**: 0.91
- **Kappa**: 0.82
- **Mean Cross-Val F1**: 0.928

**Neutro vs Positivo**
- **Precision**: 0.94
- **Recall**: 0.94
- **F1-Score**: 0.94
- **Accuracy**: 0.94
- **Kappa**: 0.88
- **Mean Cross-Val F1**: 0.908

### GloVe de 50 Dimensões

**Negativo vs Não Negativo**
- **Precision**: 0.91
- **Recall**: 0.91
- **F1-Score**: 0.91
- **Accuracy**: 0.91
- **Kappa**: 0.83
- **Mean Cross-Val F1**: 0.897

**Neutro vs Positivo**
- **Precision**: 0.94
- **Recall**: 0.94
- **F1-Score**: 0.94
- **Accuracy**: 0.94
- **Kappa**: 0.89
- **Mean Cross-Val F1**: 0.912


### Comparação de Desempenho

#### FastText

O modelo FastText apresenta os melhores resultados em quase todas as métricas para ambas as classificações:
- **Negativo vs Não Negativo**: Apresenta uma precisão, recall, f1-score e acurácia de 0.94, além de um kappa de 0.88. A média de validação cruzada do f1-score é 0.933 com um desvio padrão de 0.018.
- **Neutro vs Positivo**: Também lidera com precisão, recall, f1-score e acurácia de 0.97, e um kappa de 0.93. A média de validação cruzada do f1-score é 0.931 com um desvio padrão de 0.006.

#### TF-IDF

O modelo TF-IDF também apresenta bons resultados, mas é ligeiramente inferior ao FastText:
- **Negativo vs Não Negativo**: A precisão é 0.92, recall é 0.91, f1-score é 0.91, acurácia é 0.91 e o kappa é 0.82. A média de validação cruzada do f1-score é 0.928.
- **Neutro vs Positivo**: A precisão, recall, f1-score e acurácia são todas 0.94, com um kappa de 0.88. A média de validação cruzada do f1-score é 0.908.

#### GloVe de 50 Dimensões

O modelo GloVe de 50 dimensões mostra desempenho similar ao TF-IDF:
- **Negativo vs Não Negativo**: A precisão, recall, f1-score e acurácia são todas 0.91, com um kappa de 0.83. A média de validação cruzada do f1-score é 0.897.
- **Neutro vs Positivo**: A precisão, recall, f1-score e acurácia são todas 0.94, com um kappa de 0.89. A média de validação cruzada do f1-score é 0.912.

### Conclusão

Com base nas métricas apresentadas, o **FastText** é o melhor modelo para as classificações Negativo vs Não Negativo e Neutro vs Positivo. Ele apresenta as melhores métricas de precisão, recall, f1-score, acurácia e kappa, indicando um desempenho superior na classificação de sentimentos.


